In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_df=pd.read_csv('../input/bike-sharing-demand/train.csv')
test_df=pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
train_df.head(3)

In [ ]:
test_df.head(3)

In [ ]:
train_df.info()

In [ ]:
plt.figure(figsize=(16,8))
sns.histplot(train_df['count'],kde = True)
plt.show()

In [ ]:
def extract_features(df):
    df.datetime = pd.to_datetime(df.datetime)
    df['hour'] = df.datetime.dt.hour
    df['month'] = df.datetime.dt.month
    df['weekday'] = df.datetime.dt.weekday

In [ ]:
extract_features(train_df)
extract_features(test_df)

In [ ]:
cor_mat= train_df[:].corr()
mask = np.array(cor_mat)
mask[np.tril_indices_from(mask)] = False
fig=plt.gcf()
fig.set_size_inches(30,12)
sns.heatmap(data=cor_mat,mask=mask,square=True,annot=True,cbar=True)

In [ ]:
train_df.drop(['datetime','casual','registered','atemp'],axis=1,inplace=True)

date_time = test_df.datetime
test_df.drop(['datetime','atemp'],axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

In [ ]:
#Since the count is skewed we use log
train_df['count']=np.log(train_df['count']+1)

Y_train = train_df['count']
X_train = train_df.drop(['count'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(X_train , Y_train,  train_size=0.8, test_size=0.2, random_state =0)

In [ ]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor( colsample_bytree = 0.8, learning_rate = 0.149,min_child_weight =25, max_depth = 3,  n_estimators = 450)
xg_reg.fit(x_train,y_train)
pred=xg_reg.predict(x_test)
sc=np.sqrt(mean_squared_error(y_train , xg_reg.predict(x_train)))
score=np.sqrt(mean_squared_error(y_test, pred))
print("The score is:",score)
print("The score is:",sc)

In [ ]:
final_y_test = xg_reg.predict(test_df) 
final_y_test = np.round(np.exp(final_y_test)-1)

In [ ]:
output = pd.DataFrame({'datetime': date_time,
                       'count': final_y_test})
output.to_csv('submission.csv', index=False)

In [ ]:
output